In [1]:
import pandas as pd
import math
import numpy as np

### Загрузка и обработка данных

In [76]:
df=pd.read_csv('Data.csv')

In [117]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y-%m-%d')
df['InvoiceDate'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d')

In [118]:
df2=df[df.InvoiceDate.str.contains(r'2010-12-\d\d', regex= True, na=False)]

In [99]:
# df2=df[df['InvoiceDate']=='2010-12-23']

In [136]:
df2['Total']=df2['Quantity']*df2['Price']

C:\Users\12967\AppData\Local\Temp\ipykernel_8480\615049507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Total']=df2['Quantity']*df2['Price']


In [137]:
df_group=df2.groupby(['Customer ID', 'Country']).agg({'Total': ['sum']}). reset_index()
df_group.columns = df_group.columns.droplevel(1)

In [138]:
df_group=df_group[df_group['Country']!='United Kingdom']

### Полное совместное распределение

Интересующее событие - стоимость чека декабря 2010 года,
Событие наблюдение - страна 

In [139]:
df_group['Max_Price']=(df_group['Total']//100+1)*100
df_group['Max_Price'] = df_group['Max_Price'].astype(int) 

In [140]:
country=df_group['Country'].unique()
max_price=df_group['Max_Price'].unique()

In [141]:
df_country=pd.DataFrame(data=country, columns=['country'])
df_country['key']=0

In [142]:
df_max_price=pd.DataFrame(data=max_price, columns=['max_price'])
df_max_price['key']=0

In [143]:
df_prob=df_country.merge(df_max_price, on='key', how='outer')
df_prob['probability']=0

In [144]:
len_df=len(df_group)

In [145]:
def count_row(country, max_price):
    df_temp=df_group[(df_group['Country']==country)&(df_group['Max_Price']==max_price)]
    return len(df_temp)

In [146]:
for i, row in df_prob.iterrows():
    country_temp=row['country']
    max_price_temp=row['max_price']
    res=count_row(country_temp, max_price_temp)
    df_prob.iat[i, 3] = res/len_df

In [147]:
df_prob=df_prob[['country', 'max_price', 'probability']].sort_values('probability', ascending=False)
df_prob

,country,max_price,probability
239,France,200,0.064103
169,Germany,0,0.051282
163,Germany,300,0.038462
170,Germany,200,0.038462
234,France,1100,0.038462
...,...,...,...
201,Denmark,500,0.000000
200,Denmark,-200,0.000000
199,Denmark,1200,0.000000
198,Denmark,1400,0.000000


### Частное распределение

#### P(country)

In [148]:
df_P_c=df_prob.groupby(['country']).agg({'probability': ['sum']}). reset_index()
df_P_c.columns = df_P_c.columns.droplevel(1)
df_P_c

,country,probability
0,Australia,0.038462
1,Austria,0.025641
2,Belgium,0.051282
3,Channel Islands,0.012821
4,Cyprus,0.012821
5,Denmark,0.012821
6,EIRE,0.038462
7,Finland,0.012821
8,France,0.192308
9,Germany,0.269231


#### P(max_price)

In [150]:
df_P_p=df_prob.groupby(['max_price']).agg({'probability': ['sum']}). reset_index()
df_P_p.columns = df_P_p.columns.droplevel(1)
df_P_p

,max_price,probability
0,-200,0.012821
1,0,0.089744
2,100,0.025641
3,200,0.179487
4,300,0.128205
5,400,0.076923
6,500,0.051282
7,700,0.051282
8,800,0.051282
9,900,0.051282


### Условное распределение

#### P(max_price>500 | country = Germany)

In [152]:
P_more500_and_UK=df_prob[(df_prob['max_price']>500)&(df_prob['country']=='Germany')]['probability'].sum()

In [153]:
P_UK=df_prob[df_prob['country']=='Germany']['probability'].sum()

In [154]:
P_more500_UK=P_more500_and_UK/P_UK
P_more500_UK

0.42857142857142855